In [2]:
import pandas as pd 
df=pd.read_csv('../data/cleaned/merged_atm_data.csv')

/tmp/ipykernel_97/2304650500.py:1: DtypeWarning: Columns (2,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/kaggle/input/atm-data/merged_atm_data.csv')


In [3]:
df.head()

,atm_id,date,amount,BRANCH_NAME,CITY_CODE,city,city.1,region,day_off_discription
0,1,2009-01-02,17200,zerktouni,780,casablanca,Casablanca,casablanca-settat,NaN
1,2,2009-01-02,163300,zerktouni,780,casablanca,Casablanca,casablanca-settat,NaN
2,3,2009-01-02,306900,centre d'affaire trabless,810,rabat,Rabat,rabat-sale-kenitra,NaN
3,4,2009-01-02,231400,mohammed v,810,rabat,Rabat,rabat-sale-kenitra,NaN
4,5,2009-01-02,241000,maamora,330,kenitra,Kenitra,rabat-sale-kenitra,NaN


In [4]:
df.columns

Index(['atm_id', 'date', 'amount', 'BRANCH_NAME', 'CITY_CODE', 'city',
       'city.1', 'region', 'day_off_discription'],
      dtype='object')

In [5]:
df=df.drop(columns=['city.1'])

In [6]:
df.dtypes

atm_id                  int64
date                   object
amount                 object
BRANCH_NAME            object
CITY_CODE               int64
city                   object
region                 object
day_off_discription    object
dtype: object

In [23]:
df['date']=pd.to_datetime(df['date'])

In [8]:
# Extract the day of the year (1-365/366), useful for capturing seasonal trends
df['dayofyear'] = df['date'].dt.dayofyear

# Apply sine transformation to encode cyclic nature of the year (seasonality)
df['sin_day'] = np.sin(2 * np.pi * df['dayofyear'] / 365.25)

# Apply cosine transformation to complement sine and preserve angular distance
df['cos_day'] = np.cos(2 * np.pi * df['dayofyear'] / 365.25)

# Extract the day of the week (0=Monday, 6=Sunday) to capture weekly patterns
df['day_of_week'] = df['date'].dt.dayofweek

# Extract the day of the month (1-31), which may help if there are end/beginning-of-month effects
df['day_of_month'] = df['date'].dt.day

# Extract the month of the year (1=Jan, 12=Dec), helpful for detecting seasonal cycles
df['month_of_year'] = df['date'].dt.month

# Extract the ISO week number of the year (1-52/53), may help in capturing weekly seasonality
df['week_of_year'] = df['date'].dt.isocalendar().week

# Binary flag indicating if it's the end of the month; can capture patterns in cash demand near month-end
df['is_end_of_month'] = (df['date'].dt.is_month_end).astype(int)

# Binary flag indicating if it's the start of the month (first 3 days); useful if people withdraw early-month
df['is_start_of_month'] = (df['day_of_month'] <= 3).astype(int)

# Number of days remaining until the end of the current month
df['days_until_month_end'] = df['date'].dt.days_in_month - df['day_of_month']


In [9]:
# Create a new categorical feature 'weekend_type' to classify days based on their proximity to the weekend
df['weekend_type'] = np.select(
    [
        df['day_of_week'] == 4,                    # If it's Friday (pre-weekend)
        df['day_of_week'].isin([5, 6])             # If it's Saturday or Sunday (weekend)
    ],
    [
        'pre_weekend',                             # Label Friday as 'pre_weekend'
        'weekend'                                  # Label Saturday and Sunday as 'weekend'
    ],
    default='workday'                              # Label all other days as 'workday'
)


In [10]:
# Fill missing values in 'day_off_discription' with 'normal day' to indicate no special event or holiday
df['day_off_discription'] = df['day_off_discription'].fillna('normal day')

In [11]:
# Replace specific non-holiday technical event descriptions with 'normal day'
# These events (e.g., migrations) are likely operational and not relevant to ATM cash demand
df['day_off_discription'] = df['day_off_discription'].replace({
    'migrations  du db2 et du cics': 'normal day',
    'migration z9 vers le z10': 'normal day',
    'migration des agences chaouia': 'normal day',
})

In [12]:
# Define a list of known holidays (religious, national, historical)
holidays = ["manifeste de l'independance", 'aid al adha', 'aid al fitr', 'aid al mawlid', 'premier moharram',
           'fete du trone', 'fete du travail', 'jour de l\'an', "fete de l'independance", 'marche verte', 
           'revolution du roi et du peuple', 'fete de la jeunesse', 'fete oued eddahab', 'nouvel an amazigh']

# Create a binary flag indicating if the day is a holiday
df['is_holiday'] = df['day_off_discription'].isin(holidays).astype(int)


In [13]:
# Function to compute the number of days since the last holiday for each date
def add_days_after_last_holiday(df):
    result_df = df.copy()  # Work on a copy to avoid modifying the original DataFrame
    
    result_df = result_df.sort_index()  # Ensure data is sorted by date for correct time difference calculations

    # If no holidays exist in the data, create a column filled with NaNs and exit
    if sum(result_df['is_holiday'] == 1) == 0:
        print("No holidays found in the dataset")
        result_df['days_after_last_holiday'] = np.nan
        return result_df

    # Ensure the DataFrame index is in datetime format
    if not pd.api.types.is_datetime64_dtype(result_df.index):
        result_df.index = pd.to_datetime(result_df.index)

    # Initialize the new column
    result_df['days_after_last_holiday'] = np.nan
    
    last_holiday = None  # Variable to keep track of the last holiday date

    # Iterate through each date to compute days since the last holiday
    for date in result_df.index:
        is_holiday_val = result_df.loc[date, 'is_holiday']

        # Handle the case where loc returns a Series (e.g., duplicate index)
        if isinstance(is_holiday_val, pd.Series):
            is_holiday_val = is_holiday_val.iloc[0]
        
        # Update last holiday date if current day is a holiday
        if is_holiday_val == 1:
            last_holiday = date

        # Compute days since the last holiday if there was one
        if last_holiday is not None:
            delta = date - last_holiday
            result_df.loc[date, 'days_after_last_holiday'] = delta.days
    
    return result_df

# Apply the function to the main DataFrame
df = add_days_after_last_holiday(df)

In [14]:
#for memory restrictions , and we will work just with 3 years after
df = df[df['date'] >= '2021-01-01']

In [21]:
# Step 3: Infer an estimated urban zone type for each city
# List of 41 major Moroccan cities considered as "urban" (standardized to lowercase without accents)
urban_cities = [
    'casablanca', 'fes', 'tanger', 'marrakech', 'meknes', 'agadir', 'rabat', 'oujda', 'sale',
    'kenitra', 'tetouan', 'temara', 'safi', 'mohammedia', 'laayoune', 'el jadida', 'khouribga',
    'beni mellal', 'nador', 'guelmim', 'settat', 'khemisset', 'taza', 'larache', 'ait melloul',
    'al hoceima', 'bouskoura', 'inezgane', 'berkane', 'khenifra', 'taourirt', 'ben guerir',
    'dakhla', 'dcheira el jihadia', 'errachidia', 'sidi slimane', 'tiznit', 'taroudant',
    'guercif', 'essaouira', 'sidi kacem'
]

# Create a new column 'zone_type' based on city membership in the urban list
# Assumes 'city' column is already in lowercase and normalized (no accents)
df['zone_type'] = df['city'].apply(
    lambda x: 'urbain' if x in urban_cities else 'rural'
)


In [24]:
# Generic function to create temporal indicators (binary flags) around a specific event
def create_event_window(df, event_name, prefix, days_before=0, days_after=0):
    event_dates = df[df['day_off_discription'] == event_name]['date'].unique()
    for i in range(-days_before, days_after + 1):
        col = f"{prefix}_D{i}" if i != 0 else f"{prefix}_day"
        df[col] = df['date'].isin([d + pd.Timedelta(days=i) for d in event_dates]).astype(int)
    return df

# 1. Aid al-Adha: significant impact from 15 days before to 10 days after; peak at D-1, very low on D0
df = create_event_window(df, 'aid al adha', 'AidAdha', days_before=15, days_after=10)

# 2. Aid al-Fitr: impact from 7 days before to 3 days after
df = create_event_window(df, 'aid al fitr', 'AidFitr', days_before=7, days_after=3)

# 3. Mawlid (Prophet's birthday): impact from 2 days before to 2 days after
df = create_event_window(df, 'aid al mawlid', 'Mawlid', days_before=2, days_after=2)

# 4. 1st Moharram (Islamic New Year): impact from 1 day before to 1 day after
df = create_event_window(df, 'premier moharram', 'Hijri', days_before=1, days_after=1)

# 5. Labour Day (May 1st): impact from 1 day before to the same day
df = create_event_window(df, 'fete du travail', 'FeteTravail', days_before=1, days_after=0)

# 6. Throne Day: impact from 1 day before to 1 day after
df = create_event_window(df, 'fete du trone', 'FeteTrone', days_before=1, days_after=1)

# 7. Revolution of the King and the People + Youth Day (Aug 20–21): impact from 2 days before to 1 day after
df = create_event_window(df, 'revolution du roi et du peuple', 'Revolution20aout', days_before=2, days_after=1)
df = create_event_window(df, 'fete de la jeunesse', 'Jeunesse21aout', days_before=1, days_after=1)

# 8. Green March (Nov 6): impact from 1 day before to 1 day after
df = create_event_window(df, 'marche verte', 'MarcheVerte', days_before=1, days_after=1)

# 9. Independence Day (Nov 18): impact from 1 day before to 1 day after
df = create_event_window(df, 'fete de l\'independance', 'FeteIndependance', days_before=1, days_after=1)

# 10. Manifesto of Independence (Jan 11): impact from 1 day before to the same day
df = create_event_window(df, 'manifeste de l\'independance', 'ManifesteIndep', days_before=1, days_after=0)

# 11. New Year's Day (Jan 1): impact from 1 day before to the same day
df = create_event_window(df, 'jour de l\'an', 'JourAn', days_before=1, days_after=0)

# 12. Oued Eddahab Day: low effect, only mark the exact day
df['FeteOuedEddahab_day'] = (df['day_off_discription'] == 'fete oued eddahab').astype(int)

# 13. Amazigh New Year: minor effect, only mark the exact day
df['NouvelAnAmazigh_day'] = (df['day_off_discription'] == 'nouvel an amazigh').astype(int)

# 14. Create general features for whether today or tomorrow is a holiday
df['ferie_today'] = (df['day_off_discription'] != 'normal Day').astype(int)
df['ferie_tomorrow'] = df['date'].isin(df[df['day_off_discription'] != 'normal Day']['date'] - pd.Timedelta(days=1)).astype(int)

# Quick check: summarize and print the counts of certain event-related columns
event_cols = [col for col in df.columns if any(key in col for key in ['AidAdha', 'AidFitr', 'ferie', 'Revolution', 'FeteTravail'])]
print(df[event_cols].sum().sort_values(ascending=False))

ferie_today             2656921
ferie_tomorrow          2653271
AidFitr_D-7               18049
AidFitr_D-6               17959
AidFitr_D-3               17954
AidFitr_D-2               17810
AidFitr_D-4               17789
AidFitr_D3                17773
AidFitr_D-5               17748
AidFitr_D2                17392
AidFitr_D-1               17153
AidFitr_D1                16816
AidFitr_day               16355
AidAdha_D-6               14599
AidAdha_D-13              14588
AidAdha_D8                14524
AidAdha_D7                14521
AidAdha_D-7               14502
AidAdha_D-14              14499
AidAdha_D4                14477
AidAdha_D-10              14431
AidAdha_D5                14430
AidAdha_D-12              14413
AidAdha_D-2               14407
AidAdha_D-3               14389
AidAdha_D-9               14370
AidAdha_D6                14349
AidAdha_D-11              14326
AidAdha_D9                14290
AidAdha_D-8               14283
AidAdha_D-5               14246
AidAdha_

In [27]:
# Replace comma with dot and convert to float
df['amount'] = df['amount'].astype(str).str.replace(',', '.').astype(float)

In [28]:
# Sort the dataframe by atm_id and date to ensure proper time series order
df = df.sort_values(['atm_id', 'date'])

# Calculate rolling (moving) averages and standard deviation over 7 and 30 days per ATM
df['mm7'] = df.groupby('atm_id')['amount'].transform(
    lambda x: x.rolling(7, min_periods=1).mean()
)  # 7-day moving average

df['mm30'] = df.groupby('atm_id')['amount'].transform(
    lambda x: x.rolling(30, min_periods=1).mean()
)  # 30-day moving average

df['std7'] = df.groupby('atm_id')['amount'].transform(
    lambda x: x.rolling(7, min_periods=1).std()
)  # 7-day rolling standard deviation

# Coefficient of variation over 7 days: std / mean (adding small epsilon to avoid division by zero)
df['cv7'] = df['std7'] / (df['mm7'] + 1e-6)

# Calculate 14-day trend: difference between average of last 7 days and first 7 days in a rolling window
def trend_14d(x):
    return x.rolling(14, min_periods=1).apply(
        lambda y: y[-7:].mean() - y[:7].mean() if len(y) == 14 else 0
    )

df['trend7'] = df.groupby('atm_id')['amount'].transform(trend_14d)

# Create lagged features for previous days' amounts (lags 1 to 7, 14, and 30 days)
df['lag_1'] = df.groupby('atm_id')['amount'].shift(1)
df['lag_2'] = df.groupby('atm_id')['amount'].shift(2)
df['lag_3'] = df.groupby('atm_id')['amount'].shift(3)
df['lag_4'] = df.groupby('atm_id')['amount'].shift(4)
df['lag_5'] = df.groupby('atm_id')['amount'].shift(5)
df['lag_6'] = df.groupby('atm_id')['amount'].shift(6)
df['lag_7'] = df.groupby('atm_id')['amount'].shift(7)
df['lag_14'] = df.groupby('atm_id')['amount'].shift(14)
df['lag_30'] = df.groupby('atm_id')['amount'].shift(30)

In [29]:
# Calculate the average withdrawal amount on weekends (Saturday=5, Sunday=6) per ATM
weekend_avg = df[df['day_of_week'].isin([5, 6])].groupby('atm_id')['amount'].mean()

# Calculate the average withdrawal amount on weekdays (non-weekend days) per ATM
weekday_avg = df[~df['day_of_week'].isin([5, 6])].groupby('atm_id')['amount'].mean()

# Compute the ratio of weekend average to weekday average, adding a small epsilon to avoid division by zero
weekend_weekday_ratio = (weekend_avg / (weekday_avg + 1e-6)).reset_index()

# Rename columns for clarity
weekend_weekday_ratio.columns = ['atm_id', 'weekend_weekday_ratio']

# Merge this ratio back into the original dataframe, matching on atm_id
df = df.merge(weekend_weekday_ratio, on='atm_id', how='left')

In [30]:
# Calculate the rolling minimum withdrawal amount over the past 30 days for each ATM
df['min30'] = df.groupby('atm_id')['amount'].transform(
    lambda x: x.rolling(30, min_periods=1).min()
)

# Calculate the rolling maximum withdrawal amount over the past 30 days for each ATM
df['max30'] = df.groupby('atm_id')['amount'].transform(
    lambda x: x.rolling(30, min_periods=1).max()
)


In [34]:
df['date']=df['date'].astype(int)

In [35]:
# Number of low withdrawal days (< 1000 MAD) over the past 30 days for each ATM
df['nb_jours_faibles30'] = df.groupby('atm_id')['amount'].transform(
    lambda x: x.rolling(30, min_periods=1).apply(lambda y: (y < 1000).sum())
)

# Daily total amount withdrawn per city
df['total_city_day'] = df.groupby(['city', 'date'])['amount'].transform('sum')

# Ratio of ATM withdrawal amount relative to the city total for the day
df['atm_ratio_city'] = df['amount'] / (df['total_city_day'] + 1e-6)

# Daily total amount withdrawn per region (note: grouping by 'region' and 'amount' might be incorrect; see below)
df['total_region_day'] = df.groupby(['region', 'amount'])['amount'].transform('sum')

# Ratio of ATM withdrawal amount relative to the region total for the day
df['atm_ratio_region'] = df['amount'] / (df['total_region_day'] + 1e-6)

# Daily total amount withdrawn per branch (note: grouping by 'BRANCH_NAME' and 'date' but summing 'date' column seems off; see below)
df['total_branch_day'] = df.groupby(['BRANCH_NAME', 'date'])['date'].transform('sum')

# Ratio of ATM withdrawal amount relative to the branch total for the day
df['atm_ratio_branch'] = df['amount'] / (df['total_branch_day'] + 1e-6)

# Daily total amount withdrawn nationwide
df['total_global_day'] = df.groupby('date')['amount'].transform('sum')

# Ratio of ATM withdrawal amount relative to the national total for the day
df['atm_relative_to_global'] = df['amount'] / (df['total_global_day'] + 1e-6)

# Rank of each ATM within its city on a given day, based on withdrawal amount (descending order)
df['atm_rank_city'] = df.groupby(['city', 'date'])['amount'].rank(method='min', ascending=False)

In [37]:
df['date']=pd.to_datetime(df['date'])

In [39]:
# Function to identify if the ATM's amount is in the top 10% within its city on a given day
def is_top_10(group):
    threshold = np.percentile(group['amount'], 90)  # 90th percentile threshold
    return (group['amount'] >= threshold).astype(int)  # Mark as 1 if above or equal, else 0

# Apply the function to each group defined by city and date, then assign results to a new column
df['atm_is_top10_city'] = df.groupby(['city', 'date']).apply(is_top_10).reset_index(drop=True)

# Compute 7-day rolling mean of 'mm7' feature for each city
df['mm7_city'] = df.groupby('city')['mm7'].transform(lambda x: x.rolling(7, min_periods=1).mean())

# Compute the ratio of ATM’s 7-day moving average to the city’s 7-day moving average
df['atm_ratio_city_mm7'] = df['mm7'] / (df['mm7_city'] + 1e-6)


/tmp/ipykernel_97/2646777334.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['atm_is_top10_city'] = df.groupby(['city', 'date']).apply(is_top_10).reset_index(drop=True)


In [40]:
import pandas as pd
import numpy as np

# Define thresholds based on the data distribution (you can tune these)
HIGH_DEPOSIT_THRESHOLD = df['amount'].quantile(0.90)  # Threshold for top 10% deposits
LOW_DEPOSIT_THRESHOLD = df['amount'].quantile(0.10)   # Threshold for bottom 10% deposits
SPIKE_THRESHOLD = df['amount'].mean() * 1.5           # Threshold for spikes (1.5 times the mean amount)

# Initialize new columns for gap features with default zero values
df['rolling_max_gap'] = 0          # Days since last high deposit
df['rolling_min_gap'] = 0          # Days since last low deposit
df['days_since_last_spike'] = 0    # Days since last spike event

# Function to calculate gap features for each ATM's time series
def compute_gaps(group):
    max_gap = []   # To store days since last high deposit
    min_gap = []   # To store days since last low deposit
    spike_gap = [] # To store days since last spike

    last_high = None   # Last date when deposit was high
    last_low = None    # Last date when deposit was low
    last_spike = None  # Last date when deposit was a spike

    # Iterate over each row in the ATM group (sorted by date)
    for _, row in group.iterrows():
        current_date = row['date']
        amount = row['amount']

        # Update last_high if current amount is high
        if amount >= HIGH_DEPOSIT_THRESHOLD:
            last_high = current_date
        # Calculate days since last high deposit or NaN if none before
        max_gap.append((current_date - last_high).days if last_high else np.nan)

        # Update last_low if current amount is low
        if amount <= LOW_DEPOSIT_THRESHOLD:
            last_low = current_date
        # Calculate days since last low deposit or NaN if none before
        min_gap.append((current_date - last_low).days if last_low else np.nan)

        # Update last_spike if current amount is a spike
        if amount >= SPIKE_THRESHOLD:
            last_spike = current_date
        # Calculate days since last spike or NaN if none before
        spike_gap.append((current_date - last_spike).days if last_spike else np.nan)

    # Assign the computed gap lists back to the group's new columns
    group['rolling_max_gap'] = max_gap
    group['rolling_min_gap'] = min_gap
    group['days_since_last_spike'] = spike_gap

    return group

# Apply the gap computation function to each ATM group in the dataframe
df = df.groupby('atm_id').apply(compute_gaps)

/tmp/ipykernel_97/1905830804.py:47: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df= df.groupby('atm_id').apply(compute_gaps)


In [41]:
# Replace NaN values in the gap columns with 0
df['rolling_max_gap'] = df['rolling_max_gap'].fillna(0)
df['rolling_min_gap'] = df['rolling_min_gap'].fillna(0)
df['days_since_last_spike'] = df['days_since_last_spike'].fillna(0)


/tmp/ipykernel_97/3283717924.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['rolling_max_gap'].fillna(0, inplace=True)
/tmp/ipykernel_97/3283717924.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using '

In [48]:
df= df.reset_index(level='atm_id', drop=True)


In [50]:
# Calculate average amount per ATM and day of week
df['avg_by_dow'] = df.groupby(['atm_id', 'day_of_week'])['amount'].transform('mean')

In [51]:
# Calculate 7-day lag correlation of amount per ATM
df['lag_corr_7'] = df.groupby('atm_id')['amount'].transform(
    lambda x: x.shift(7).corr(x)
)

In [52]:
# Obtenir la liste des colonnes de type object (catégorielles ou string)
object_features = df.select_dtypes(include='object').columns.tolist()

# Affichage
print("Colonnes de type object :", object_features)

Colonnes de type object : ['BRANCH_NAME', 'city', 'region', 'day_off_discription', 'weekend_type', 'zone_type']


In [54]:
# Drop columns 'BRANCH_NAME' and 'city' as their information is already captured elsewhere
df = df.drop(columns=['BRANCH_NAME', 'city'])

# Use label encoding for categorical columns where ordinal encoding is acceptable
from sklearn.preprocessing import LabelEncoder

for col in ['region', 'zone_type', 'day_off_discription', 'weekend_type']:
    df[col] = LabelEncoder().fit_transform(df[col])

In [55]:
df.head()

,atm_id,date,amount,CITY_CODE,region,day_off_discription,dayofyear,sin_day,cos_day,day_of_week,...,atm_relative_to_global,atm_rank_city,atm_is_top10_city,mm7_city,atm_ratio_city_mm7,rolling_max_gap,rolling_min_gap,days_since_last_spike,avg_by_dow,lag_corr_7
0,1,2021-01-01,21700.0,780,12,8,1,0.017202,0.999852,4,...,0.000067,225.0,0,21700.000000,1.000000,0.0,0.0,0.0,52822.435071,0.223148
1,1,2021-01-02,25000.0,780,12,11,2,0.034398,0.999408,5,...,0.000079,217.0,1,22525.000000,1.036626,0.0,0.0,0.0,27622.681014,0.223148
2,1,2021-01-03,13100.0,780,12,11,3,0.051584,0.998669,6,...,0.000060,206.0,0,21661.111111,0.920236,0.0,0.0,0.0,14087.452791,0.223148
3,1,2021-01-04,59500.0,780,12,11,4,0.068755,0.997634,0,...,0.000137,231.0,1,23702.083333,1.258328,0.0,1.0,0.0,52893.875167,0.223148
4,1,2021-01-05,47200.0,780,12,11,5,0.085906,0.996303,1,...,0.000130,232.0,0,25621.666667,1.299681,0.0,0.0,0.0,52377.701053,0.223148


In [61]:
df=df.reset_index()

In [62]:
df = df[df['date'] >= '2022-01-01']

In [63]:
df=df.drop_duplicates(subset=['atm_id','date'])

In [64]:
df=df.set_index('date')

In [65]:
df.to_csv('/kaggle/working/final_atm_data.csv')